# Group THE BOYS 1D ML project
1. Loy Pek Yong 1004475
2. Lim Yongqing 1005955
3. Sim Reynard Simano 1006307

# Question 1

## Importing libraries + reading data + preprocessing

In [186]:
# import libraries
import pandas as pd
import numpy as np
print(np.__version__)
print(pd.__version__)

with open('./ES/train', 'r', encoding = 'utf8') as f:
    r1 = [lines.rstrip("\n") for lines in f.readlines()]
ES_train_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r1]

with open('./ES/dev.in', 'r', encoding = 'utf8') as f:
    r2 = [lines.rstrip("\n") for lines in f.readlines()]
ES_dev_in_tup = [ lines if lines != "" else "null" for lines in r2]

with open('./ES/dev.out', 'r', encoding = 'utf8') as f:
    r3 = [lines.rstrip("\n") for lines in f.readlines()]
ES_dev_out_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r3]



with open('./RU/train', 'r', encoding = 'utf8') as f:
    r4 = [lines.rstrip("\n") for lines in f.readlines()]
RU_train_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r4]

with open('./RU/dev.in', 'r', encoding = 'utf8') as f:
    r5 = [lines.rstrip("\n") for lines in f.readlines()]
RU_dev_in_tup = [ lines if lines != "" else "null" for lines in r5]

with open('./RU/dev.out', 'r', encoding = 'utf8') as f:
    r6 = [lines.rstrip("\n") for lines in f.readlines()]
RU_dev_out_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r6]


1.23.5
1.5.3


## Changing to dataframe

In [187]:
ES_train = pd.DataFrame(data = ES_train_tup, columns = ["word", "label"])
ES_dev_in = pd.DataFrame(data = ES_dev_in_tup, columns = ["word"])
ES_dev_out = pd.DataFrame(data = ES_dev_out_tup, columns = ["word", "label"])

RU_train = pd.DataFrame(data = RU_train_tup, columns = ["word", "label"])
RU_dev_in = pd.DataFrame(data = RU_dev_in_tup, columns = ["word"])
RU_dev_out = pd.DataFrame(data = RU_dev_out_tup, columns = ["word", "label"])

# Part 1 of q1

## function that returns the emission parameters with input of training set

In [188]:
def train_estimate_emission_parameters(train):
    # word_column = train["word"]
    # label_column = train["label"]

    label_count = {}
    emission_count = {}

    for row in range(train.shape[0]):
        word = train.iloc[row, 0]
        label = train.iloc[row, 1]
        if label not in label_count:
            label_count[label] = 1
        else:
            label_count[label] += 1

        if (word, label) not in emission_count:
            emission_count[(word, label)] = 1
        else:
            emission_count[(word, label)] += 1

    emission_parameter = {}
    for word, label in emission_count:
        emission_parameter[(word, label)] = emission_count[(word, label)] / label_count[label]

    return emission_parameter, emission_count, label_count


# Part 2 of q1

## function that returns the emission parameters with input set

In [189]:
def upgraded_estimate_emission_parameters(test_set, emission_parameter, emission_counts, label_counts):
    # predict the label of the test set
    k = 0
    special_word = '#UNK#'
    test_copy = test_set.copy()
    #train word list is all the first argument in keys of emission counts
    train_word_list= [word for word, label in emission_counts.keys()]
    for i in range(len(test_copy["word"])):
        word = test_copy["word"][i]
        if word not in train_word_list:
            test_copy["word"][i] = special_word
            k+=1

    for word in test_copy["word"]:
        for label in label_counts:
            if word == special_word:
                emission_parameter[(word, label)] = k / (label_counts[label] + k)
            elif ( (word, label) in emission_parameter ):
                emission_parameter[(word, label)] = emission_counts[(word, label)] / (label_counts[label] + k)
                
    # print(emission_parameter)        
    return emission_parameter

In [190]:
emission_parameter, emission_count, label_count = train_estimate_emission_parameters(ES_train)

In [191]:
label_count

{'O': 29035,
 'B-positive': 1160,
 'null': 1857,
 'B-negative': 381,
 'B-neutral': 72,
 'I-neutral': 43,
 'I-positive': 314,
 'I-negative': 171}

In [192]:
# ES_train
filtered_rows = ES_train[ES_train['word'] == 'de']
filtered_rows

,word,label
162,de,O
168,de,I-neutral
181,de,O
200,de,O
247,de,O
...,...,...
32908,de,O
32913,de,O
32937,de,O
32945,de,O


In [193]:
filtered_de = filtered_rows[filtered_rows['label'] == 'O']
filtered_de

,word,label
162,de,O
181,de,O
200,de,O
247,de,O
261,de,O
...,...,...
32908,de,O
32913,de,O
32937,de,O
32945,de,O


In [194]:
emission_parameter = upgraded_estimate_emission_parameters(ES_dev_in, emission_parameter, emission_count, label_count)

In [195]:
for key, value in emission_parameter.items():
    if key[0] == "de":
        print(key, value)

('de', 'O') 0.03431655408845592
('de', 'I-neutral') 0.010660980810234541
('de', 'I-positive') 0.07702702702702703
('de', 'I-negative') 0.04690117252931323


# Part 3 of q1

In [196]:
def sentiment_analysis(training_set, test_set):
    emission_parameter, emission_count, label_count = train_estimate_emission_parameters(training_set)
    emission_parameter = upgraded_estimate_emission_parameters(test_set, emission_parameter, emission_count, label_count)
    test_set["label"] = ""

    #label equals to argmax of emission parameter
    for row in range(len(test_set)):
        word = test_set.iloc[row]["word"]
        max_val = 0
        max_label = ""
        train_word_list = [word for word, label in emission_count.keys()]
        
        for label in label_count:
            if (word, label) in emission_parameter:
                if emission_parameter[(word, label)] > max_val:
                    max_val = emission_parameter[(word, label)]
                    max_label = label
                    
            else:
                if word not in train_word_list:
                    # get max label from word #UNK#
                    if ("#UNK#", label) in emission_parameter:
                        if emission_parameter[("#UNK#", label)] > max_val:
                            max_val = emission_parameter[("#UNK#", label)]
                            max_label = label
        test_set.loc[row, "label"] = max_label
                    
    return test_set


## Predicting + saving to csv

In [197]:
ES_dev_p1_out = sentiment_analysis(ES_train, ES_dev_in)
ES_dev_p1_out_r = ES_dev_p1_out.replace('null', pd.NA)
ES_dev_p1_out_r.to_csv("./ES/dev.p1.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)
    
RU_dev_p1_out = sentiment_analysis(RU_train, RU_dev_in)
RU_dev_p1_out_r = RU_dev_p1_out.replace('null', pd.NA)
RU_dev_p1_out_r.to_csv("./RU/dev.p1.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)


# Scoring for part 1

In [198]:
import subprocess

command = ['python', r'.\ES\evalResult.py', r'.\ES\dev.out', r'.\ES\dev.p1.out']
result = subprocess.run(command, stdout=subprocess.PIPE)
print("ES result is:")
print(result.stdout.decode())

command = ['python', r'.\RU\evalResult.py', r'.\RU\dev.out', r'.\RU\dev.p1.out']
result = subprocess.run(command, stdout=subprocess.PIPE)
print("RU result is:")
print(result.stdout.decode())


ES result is:

#Entity in gold data: 229
#Entity in prediction: 1144

#Correct Entity : 181
Entity  precision: 0.1582
Entity  recall: 0.7904
Entity  F: 0.2637

#Correct Sentiment : 120
Sentiment  precision: 0.1049
Sentiment  recall: 0.5240
Sentiment  F: 0.1748

RU result is:

#Entity in gold data: 389
#Entity in prediction: 1467

#Correct Entity : 288
Entity  precision: 0.1963
Entity  recall: 0.7404
Entity  F: 0.3103

#Correct Sentiment : 174
Sentiment  precision: 0.1186
Sentiment  recall: 0.4473
Sentiment  F: 0.1875



# 

# Question 2

## Reading data + preprocessing

In [199]:
with open('./ES/train', 'r', encoding = 'utf8') as f:
    file = f.readlines()
EStrain = [lines.rstrip("\n") for lines in file]
EStrain_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else "STOP" for lines in EStrain ]

with open('./RU/train', 'r', encoding = 'utf8') as f:
    file = f.readlines()
RUtrain = [lines.rstrip("\n") for lines in file]
RUtrain_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else "STOP" for lines in RUtrain ]

# Part 1 of q2

## Creating reference transition dictionary (all transitions)

In [200]:
transit_dict = {
    ("O", "O"): 0,
    ("O", "B-positive"):0,
    ("O", "B-negative"):0,
    ("O", "B-neutral"):0,
    ("O", "I-positive"):0,
    ("O", "I-negative"):0,
    ("O", "I-neutral"):0,
    ("O", "START"):0,
    ("O", "STOP"):0,
    
    ("B-positive", "O"):0,
    ("B-positive", "B-positive"):0,
    ("B-positive", "B-negative"):0,
    ("B-positive", "B-neutral"):0,
    ("B-positive", "I-positive"):0,
    ("B-positive", "I-negative"):0,
    ("B-positive", "I-neutral"):0,
    ("B-positive", "START"):0,
    ("B-positive", "STOP"):0,
    
    
    ("B-negative", "O"):0,
    ("B-negative", "B-positive"):0,
    ("B-negative", "B-negative"):0,
    ("B-negative", "B-neutral"):0,
    ("B-negative", "I-positive"):0,
    ("B-negative", "I-negative"):0,
    ("B-negative", "I-neutral"):0,
    ("B-negative", "START"):0,
    ("B-negative", "STOP"):0,
    
    ("B-neutral", "O"):0,
    ("B-neutral", "B-positive"):0,
    ("B-neutral", "B-negative"):0,
    ("B-neutral", "B-neutral"):0,
    ("B-neutral", "I-positive"):0,
    ("B-neutral", "I-negative"):0,
    ("B-neutral", "I-neutral"):0,
    ("B-neutral", "START"):0,
    ("B-neutral", "STOP"):0,
    
    ("I-positive", "O"):0,
    ("I-positive", "B-positive"):0,
    ("I-positive", "B-negative"):0,
    ("I-positive", "B-neutral"):0,
    ("I-positive", "I-positive"):0,
    ("I-positive", "I-negative"):0,
    ("I-positive", "I-neutral"):0,
    ("I-positive", "START"):0,
    ("I-positive", "STOP"):0,
    
    ("I-negative", "O"):0,
    ("I-negative", "B-positive"):0,
    ("I-negative", "B-negative"):0,
    ("I-negative", "B-neutral"):0,
    ("I-negative", "I-positive"):0,
    ("I-negative", "I-negative"):0,
    ("I-negative", "I-neutral"):0,
    ("I-negative", "START"):0,
    ("I-negative", "STOP"):0,
    
    ("I-neutral", "O"):0,
    ("I-neutral", "B-positive"):0,
    ("I-neutral", "B-negative"):0,
    ("I-neutral", "B-neutral"):0,
    ("I-neutral", "I-positive"):0,
    ("I-neutral", "I-negative"):0,
    ("I-neutral", "I-neutral"):0,
    ("I-neutral", "START"):0,
    ("I-neutral", "STOP"):0,
    
    ("START", "O"):0,
    ("START", "B-positive"):0,
    ("START", "B-negative"):0,
    ("START", "B-neutral"):0,
    ("START", "I-positive"):0,
    ("START", "I-negative"):0,
    ("START", "I-neutral"):0,
    ("START", "START"):0,
    ("START", "STOP"):0,
    
    ("STOP", "O"):0,
    ("STOP", "B-positive"):0,
    ("STOP", "B-negative"):0,
    ("STOP", "B-neutral"):0,
    ("STOP", "I-positive"):0,
    ("STOP", "I-negative"):0,
    ("STOP", "I-neutral"):0,
    ("STOP", "START"):0,
    ("STOP", "STOP"):0,
               
}

## Function to calculate number of u (init state) to v (next state) transitions

In [201]:
"""
Assumptions made: 

1. Everything is in tuple pairs except the word "STOP" which signifies the end of a review
2. Transition probabilities from "STOP" to any other states are 0 but still included in the dictionary as required by question



Variables:
1. u --> initial state (e.g "O", "B-neutral", "I-positive" etc.)
2. v --> next state (e.g "O", "B-neutral", "I-positive" etc.)
3. hasbreak --> signifies the end of a review (boolean)

"""

def calculate_transition(data, transit_dict):
    updated_dict = transit_dict.copy()
    hasbreak = True
    for word in data:
        if ( hasbreak == True ):
            u = "START"
            
        if ( word != "STOP" and (hasbreak == True) ):
            v = word[1]
            updated_dict[(u, v)] += 1
            hasbreak = False
            
        elif ( word != "STOP" and (hasbreak == False) ):
            u = v
            v = word[1]
            updated_dict[(u, v)] += 1
        
        elif ( word == "STOP" ):
            u = v
            v = "STOP"
            updated_dict[(u, v)] += 1
            hasbreak = True
            
    return updated_dict

## Update for ES and RU

In [202]:
ES_updated_tdict = calculate_transition(EStrain_tup, transit_dict)
RU_updated_tdict = calculate_transition(RUtrain_tup, transit_dict)

## Function to convert above calculation to probability

In [203]:
def transition_param(updated_tdict):
    all_states = ['O', 'B-positive', 'B-negative', 'B-neutral', 'I-positive', 'I-negative', 'I-neutral', 'START', 'STOP']
    q_dict = {}
    temp_sum = 0
    for state in all_states:
        total = sum(value for key, value in updated_tdict.items() if key[1] == state)
        for key, value in updated_tdict.items():
            if ( (key[1] == state) and ( total != 0) ):
                q_dict[f"{state}|{key[0]}"] = value/total
            elif ( (key[1] == state) and ( total == 0) ):
                q_dict[f"{state}|{key[0]}"] = 0
    return q_dict

## *Below is the final parameters to be used*

*Format is v|u (next state given current state)*

In [204]:
ES_transit = transition_param(ES_updated_tdict)
RU_transit = transition_param(RU_updated_tdict)

# Getting emission params

In [205]:
ES_emission_parameter, ES_emission_count, ES_label_count = train_estimate_emission_parameters(ES_train)
ES_emission = upgraded_estimate_emission_parameters(ES_dev_in, ES_emission_parameter, ES_emission_count, ES_label_count)

RU_emission_parameter, RU_emission_count, RU_label_count = train_estimate_emission_parameters(RU_train)
RU_emission = upgraded_estimate_emission_parameters(RU_dev_in, RU_emission_parameter, RU_emission_count, RU_label_count)

# Read dev.in files

In [206]:
with open('./ES/dev.in', 'r', encoding = 'utf8') as f:
    file = f.readlines()
ESdev = [lines.rstrip("\n") for lines in file]
ESdev_ls = [ lines if lines != "" else "STOP" for lines in ESdev ]

with open('./RU/dev.in', 'r', encoding = 'utf8') as f:
    file = f.readlines()
RUdev = [lines.rstrip("\n") for lines in file]
RUdev_ls = [ lines if lines != "" else "STOP" for lines in RUdev ]

# Restructuring dev.in to list of sequences

In [207]:
def to_ls(dev_in_ls, transit_param):
    sequence_list = []
    sequence_xi = []
    for word in dev_in_ls:
        if ( word != "STOP"):
            sequence_xi.append(word)
        else:
            sequence_list.append(sequence_xi.copy())
            sequence_xi = []
    return sequence_list

In [208]:
ES_seq_list = to_ls(ESdev_ls, ES_transit)
RU_seq_list = to_ls(RUdev_ls, RU_transit)

## Based on previous state, get the possible next states based on the current word

In [209]:
def get_possible_states(prev_state, desired_word, emit, transit):
    filtered_keys = [key for key in emit.keys() if (key[0] == desired_word)]
    signal = False
    if ( len(filtered_keys) == 0 ):
        desired_pattern = f"|{prev_state}"
        unk = [(key, value) for key, value in transit.items() if ( key.endswith(desired_pattern) and not key.startswith("STOP") and not key.startswith("START") and value!=0 )]
        filtered_keys = [(desired_word, item[0].split('|')[0]) for item in unk]
        signal = True
    possible_states = [key[1] for key in filtered_keys]
    return possible_states, signal

# Viterbi

In [210]:
def viterbi(sequence, emit, transit, u, v, index, dt = {}):
    u = v
    dt[(0,v)] = 0
    max_u_ls = []
    possible_states = ["START"]
    word = ""
    for i in range(len(sequence)):
        save_state = possible_states
        word = sequence[index]
        signal = False
        possible_states, signal = get_possible_states(save_state[0], word, emit, transit)
        for u in save_state:
            for v in possible_states:
                a = np.log(transit[f"{v}|{u}"])
                if signal == True:
                    b = np.log(emit[('#UNK#', v)])
                else:
                    b = np.log(emit[(word, v)])
                max_u_ls.append(dt[(index, u)] + a + b)
            for j in range(len(possible_states)):
                key = ( index+1, possible_states[j])
                if key not in dt:
                    dt[key] = max_u_ls[j]
                elif dt[key] < max_u_ls[j]:
                    dt[key] = max_u_ls[j]
            max_u_ls = max_u_ls[len(possible_states):]
        max_u_ls = []
        index += 1
    dct = dt.copy()
    return dct

# Prediction function

In [211]:
def predict(emit, transit, seq_list):
    dt = {}
    pred = []
    for i in seq_list:
        temp = viterbi(i, emit, transit, "START", "START", 0, {})
        pred.append(temp)
    return pred

# Predict

#### Note: Might have divide by zero error but it's fine

In [212]:
ES_pred = predict(ES_emission, ES_transit, ES_seq_list)
RU_pred = predict(RU_emission, RU_transit, RU_seq_list)

C:\Users\limyo\AppData\Local\Temp\ipykernel_4404\4078816607.py:14: RuntimeWarning: divide by zero encountered in log
  a = np.log(transit[f"{v}|{u}"])


## For each duplicated index, look for the one with highest log value a.k.a highest prob

In [213]:
def processing_pred(pred):
    pred_ls = []
    grouped_dict = {}
    for seq in pred:
        for key, value in seq.items():
            var = key[0]
            if var in range(0, len(seq)):
                if var not in grouped_dict or value > grouped_dict[var][1]:
                    grouped_dict[var] = (key, value)

        pred_ls.append([item[0] for item in grouped_dict.values()])
        grouped_dict = {}
    return pred_ls

In [214]:
ES_proc = processing_pred(ES_pred)
RU_proc = processing_pred(RU_pred)

# Convert to df

In [215]:
def consolidate(pred_res, seq_list):
    res = pd.DataFrame(columns=['word', 'label'])
    for seq_i in range(len(seq_list)):
        labels = [label for _, label in pred_res[seq_i][1:]]
        df = pd.DataFrame({
            'word': seq_list[seq_i],
            'label': labels
        })
        df.loc[len(df)] = ''
        res = pd.concat([res, df])
    return res

    

In [216]:
ES_pred_q2 = consolidate(ES_proc, ES_seq_list)
RU_pred_q2 = consolidate(RU_proc, RU_seq_list)

# Saving to csv

In [217]:
ES_pred_q2.to_csv("./ES/dev.p2.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)
RU_pred_q2.to_csv("./RU/dev.p2.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)

# Scoring for q2

In [218]:
import subprocess

command2 = ['python', r'.\ES\evalResult.py', r'.\ES\dev.out', r'.\ES\dev.p2.out']
result2 = subprocess.run(command2, stdout=subprocess.PIPE)
print("ES result is:")
print(result2.stdout.decode())

command2 = ['python', r'.\RU\evalResult.py', r'.\RU\dev.out', r'.\RU\dev.p2.out']
result2 = subprocess.run(command2, stdout=subprocess.PIPE)
print("RU result is:")
print(result2.stdout.decode())


ES result is:

#Entity in gold data: 229
#Entity in prediction: 1014

#Correct Entity : 139
Entity  precision: 0.1371
Entity  recall: 0.6070
Entity  F: 0.2237

#Correct Sentiment : 86
Sentiment  precision: 0.0848
Sentiment  recall: 0.3755
Sentiment  F: 0.1384

RU result is:

#Entity in gold data: 389
#Entity in prediction: 1392

#Correct Entity : 234
Entity  precision: 0.1681
Entity  recall: 0.6015
Entity  F: 0.2628

#Correct Sentiment : 150
Sentiment  precision: 0.1078
Sentiment  recall: 0.3856
Sentiment  F: 0.1684



# No.3



### We will be modifying the viterbi algorithm by varying the K, obtaining the best k-th sequence

In [219]:
import pandas as pd

with open('./ES/train', 'r', encoding = 'utf8') as f:
    r1 = [lines.rstrip("\n") for lines in f.readlines()]
ES_train_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r1]

with open('./ES/dev.in', 'r', encoding = 'utf8') as f:
    r2 = [lines.rstrip("\n") for lines in f.readlines()]
ES_dev_in_tup = [ lines if lines != "" else "null" for lines in r2]

with open('./ES/dev.out', 'r', encoding = 'utf8') as f:
    r3 = [lines.rstrip("\n") for lines in f.readlines()]
ES_dev_out_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r3]



with open('./RU/train', 'r', encoding = 'utf8') as f:
    r4 = [lines.rstrip("\n") for lines in f.readlines()]
RU_train_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r4]

with open('./RU/dev.in', 'r', encoding = 'utf8') as f:
    r5 = [lines.rstrip("\n") for lines in f.readlines()]
RU_dev_in_tup = [ lines if lines != "" else "null" for lines in r5]

with open('./RU/dev.out', 'r', encoding = 'utf8') as f:
    r6 = [lines.rstrip("\n") for lines in f.readlines()]
RU_dev_out_tup = [ (lines.rsplit(" ", 1)[0], lines.rsplit(" ", 1)[1]) if lines != "" else ("null", "null") for lines in r6]

ES_train = pd.DataFrame(data = ES_train_tup, columns = ["word", "label"])
ES_dev_in = pd.DataFrame(data = ES_dev_in_tup, columns = ["word"])
ES_dev_out = pd.DataFrame(data = ES_dev_out_tup, columns = ["word", "label"])

RU_train = pd.DataFrame(data = RU_train_tup, columns = ["word", "label"])
RU_dev_in = pd.DataFrame(data = RU_dev_in_tup, columns = ["word"])
RU_dev_out = pd.DataFrame(data = RU_dev_out_tup, columns = ["word", "label"])

with open('./ES/dev.in', 'r', encoding = 'utf8') as f:
    file = f.readlines()
ESdev = [lines.rstrip("\n") for lines in file]
ESdev_ls = [ lines if lines != "" else "STOP" for lines in ESdev ]

with open('./RU/dev.in', 'r', encoding = 'utf8') as f:
    file = f.readlines()
RUdev = [lines.rstrip("\n") for lines in file]
RUdev_ls = [ lines if lines != "" else "STOP" for lines in RUdev ]

### Using these values emmision and transition parameters, we will proceed to count the sequence by repeating the viterbi algortihm we obtained in q2

In [220]:

ES_emission_parameter, ES_emission_count, ES_label_count = train_estimate_emission_parameters(ES_train)
ES_emission_2nd = upgraded_estimate_emission_parameters(ES_dev_in, ES_emission_parameter, ES_emission_count, ES_label_count)
ES_emission_8th = upgraded_estimate_emission_parameters(ES_dev_in, ES_emission_parameter, ES_emission_count, ES_label_count)

RU_emission_parameter, RU_emission_count, RU_label_count = train_estimate_emission_parameters(RU_train)
RU_emission_2nd = upgraded_estimate_emission_parameters(RU_dev_in, RU_emission_parameter, RU_emission_count, RU_label_count)
RU_emission_8th = upgraded_estimate_emission_parameters(RU_dev_in, RU_emission_parameter, RU_emission_count, RU_label_count)

ES_transit = transition_param(ES_updated_tdict)
RU_transit = transition_param(RU_updated_tdict)

ES_seq_list = to_ls(ESdev_ls, ES_transit)
RU_seq_list = to_ls(RUdev_ls, RU_transit)


### Here we will modifying the processing_pred function to take in an argument k

In [221]:
import heapq

def mod_processing_pred(pred, k):
    all_pred_ls = []
    for seq in pred:
        # Initialize priority queue with the highest value sequence
        queue = []
        grouped_dict = {}
        for key, value in seq.items():
            var = key[0]
            if var in range(0, len(seq)):
                if var not in grouped_dict or value > grouped_dict[var][1]:
                    grouped_dict[var] = (key, value)
        initial_sequence = [item[0] for item in grouped_dict.values()]
        initial_value = sum(item[1] for item in grouped_dict.values())
        heapq.heappush(queue, (-initial_value, initial_sequence))

        # Generate top 10 sequences to limit memory
        pred_ls = []
        for _ in range(10):
            if not queue:
                break
            value, sequence = heapq.heappop(queue)
            pred_ls.append(sequence)

            # Generate new sequences by replacing one element at a time
            for i in range(len(sequence)):
                replaced_key = sequence[i]
                remaining_keys = [key for key in seq.keys() if key[0] == replaced_key[0] and key != replaced_key]
                for new_key in remaining_keys:
                    new_sequence = sequence[:i] + [new_key] + sequence[i+1:]
                    new_value = value - seq[replaced_key] + seq[new_key]
                    heapq.heappush(queue, (-new_value, new_sequence))

        all_pred_ls.append(pred_ls)

    kth_sequences = [seq[min(k-1, len(seq)-1)] for seq in all_pred_ls ]

    
    return kth_sequences

In [222]:
ES_pred_2nd = predict(ES_emission_2nd, ES_transit, ES_seq_list)
RU_pred_2nd = predict(RU_emission_2nd, RU_transit, RU_seq_list)

ES_proc_2nd = mod_processing_pred(ES_pred_2nd, k=2)
RU_proc_2nd = mod_processing_pred(RU_pred_2nd, k=2)

ES_pred_k2 = consolidate(ES_proc_2nd, ES_seq_list)
RU_pred_k2 = consolidate(RU_proc_2nd, RU_seq_list)

ES_pred_k2.to_csv("./ES/dev.p3.2nd.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)
RU_pred_k2.to_csv("./RU/dev.p3.2nd.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)


C:\Users\limyo\AppData\Local\Temp\ipykernel_4404\4078816607.py:14: RuntimeWarning: divide by zero encountered in log
  a = np.log(transit[f"{v}|{u}"])
C:\Users\limyo\AppData\Local\Temp\ipykernel_4404\1238102201.py:32: RuntimeWarning: invalid value encountered in double_scalars
  new_value = value - seq[replaced_key] + seq[new_key]


In [223]:
ES_pred_8th = predict(ES_emission_8th, ES_transit, ES_seq_list)
RU_pred_8th = predict(RU_emission_8th, RU_transit, RU_seq_list)

ES_proc_8th = mod_processing_pred(ES_pred_8th, 8)
RU_proc_8th = mod_processing_pred(RU_pred_8th, 8)

ES_pred_k8 = consolidate(ES_proc_8th, ES_seq_list)
RU_pred_k8 = consolidate(RU_proc_8th, RU_seq_list)

ES_pred_k8.to_csv("./ES/dev.p3.8th.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)
RU_pred_k8.to_csv("./RU/dev.p3.8th.out", sep=" ", header=None, index=False, encoding="utf-8", quoting=3)


C:\Users\limyo\AppData\Local\Temp\ipykernel_4404\4078816607.py:14: RuntimeWarning: divide by zero encountered in log
  a = np.log(transit[f"{v}|{u}"])
C:\Users\limyo\AppData\Local\Temp\ipykernel_4404\1238102201.py:32: RuntimeWarning: invalid value encountered in double_scalars
  new_value = value - seq[replaced_key] + seq[new_key]


### Report

In [224]:
import subprocess

command3 = ['python', r'.\ES\evalResult.py', r'.\ES\dev.out', r'./ES/dev.p3.2nd.out']
result3 = subprocess.run(command3, stdout=subprocess.PIPE)
print("ES result is:")
print(result3.stdout.decode())

command3 = ['python', r'.\RU\evalResult.py', r'.\RU\dev.out', r'./RU/dev.p3.2nd.out']
result3 = subprocess.run(command3, stdout=subprocess.PIPE)
print("RU result is:")
print(result3.stdout.decode())

ES result is:

#Entity in gold data: 229
#Entity in prediction: 1044

#Correct Entity : 139
Entity  precision: 0.1331
Entity  recall: 0.6070
Entity  F: 0.2184

#Correct Sentiment : 69
Sentiment  precision: 0.0661
Sentiment  recall: 0.3013
Sentiment  F: 0.1084

RU result is:

#Entity in gold data: 389
#Entity in prediction: 1424

#Correct Entity : 230
Entity  precision: 0.1615
Entity  recall: 0.5913
Entity  F: 0.2537

#Correct Sentiment : 136
Sentiment  precision: 0.0955
Sentiment  recall: 0.3496
Sentiment  F: 0.1500



In [225]:
command3 = ['python', r'.\ES\evalResult.py', r'.\ES\dev.out', r'./ES/dev.p3.8th.out']
result3 = subprocess.run(command3, stdout=subprocess.PIPE)
print("ES result is:")
print(result3.stdout.decode())

command3 = ['python', r'.\RU\evalResult.py', r'.\RU\dev.out', r'./RU/dev.p3.8th.out']
result3 = subprocess.run(command3, stdout=subprocess.PIPE)
print("RU result is:")
print(result3.stdout.decode())

ES result is:

#Entity in gold data: 229
#Entity in prediction: 1061

#Correct Entity : 141
Entity  precision: 0.1329
Entity  recall: 0.6157
Entity  F: 0.2186

#Correct Sentiment : 75
Sentiment  precision: 0.0707
Sentiment  recall: 0.3275
Sentiment  F: 0.1163

RU result is:

#Entity in gold data: 389
#Entity in prediction: 1408

#Correct Entity : 213
Entity  precision: 0.1513
Entity  recall: 0.5476
Entity  F: 0.2371

#Correct Sentiment : 116
Sentiment  precision: 0.0824
Sentiment  recall: 0.2982
Sentiment  F: 0.1291



# Question 4

## Separate into sentences by '.', '?', '!'

In [226]:
# def split_list(lst):
#     result = []
#     sublist = []
#     for item in lst:
#         if item[0] == '.' or  item[0] == '?' or  item[0] == '!':
#             sublist.append(item)
#             result.append(sublist)
#             sublist = []
#         else:
#             sublist.append(item)
#     if sublist:
#         result.append(sublist)
#     return result

## Process ES sentences

In [227]:
# # Combine entries in tuple form
# es_sentences_combined = [tuple(x) for x in ES_train.to_records(index=False)]
# # Filter out null entries
# es_sentences_combined = list(filter(lambda x: x != ('null', 'null'), es_sentences_combined))

# es_sentences = split_list(es_sentences_combined)

## Process RU sentences

In [228]:
# # Combine entries in tuple form
# ru_sentences_combined = [tuple(x) for x in RU_train.to_records(index=False)]
# # Filter out null entries
# ru_sentences_combined = list(filter(lambda x: x != ('null', 'null'), ru_sentences_combined))

# ru_sentences = split_list(ru_sentences_combined)

## Extract ES features by word

In [229]:
# # Spanish
# import re

# def get_features_es(word):
#     features = {}
  
#     # Check last 3 letters for Spanish verb conjugation
#     features['VERB'] = int(word.endswith('o') or word.endswith('as') or word.endswith('a') or word.endswith('amos'))
#     # Check last 2 letters for Spanish plurals and gender
#     features['PLURAL'] = int(word.endswith('os') or word.endswith('as'))
#     features['MASCULINE'] = int(word.endswith('o'))
#     features['FEMININE'] = int(word.endswith('a'))

#     # Check last 4 letters for Spanish superlative
#     features['SUPERLATIVE'] = int(word.endswith('ísimo'))

#     # Check for diminutive suffixes
#     features['DIMINUTIVE'] = 1 if re.search(r'ito$|ita$|ito$|illa$|ico$|ica$|illo$|illa$', word) else 0
    
#     # Check for punctuation
#     features['has_punctuation'] = int(any(char in word for char in r'!?.,"-'))
    
#     return features

## Extract RU features by word

In [230]:
# # Russian
# def get_features_ru(word):

#     features = {}
  
#     # Check last 2 letters for verb conjugation
#     features['INFINITIVE'] = int(word.endswith('ть'))
#     features['PAST_MASC'] = int(word.endswith('л'))
#     features['PAST_FEM'] = int(word.endswith('ла'))

#     # Check last letter for noun gender 
#     features['FEMININE'] = int(word.endswith('а'))
#     features['MASCULINE'] = int(word.endswith('й'))

#     # Check last 3 letters for plural nouns
#     features['PLURAL'] = int(word.endswith('ие'))

#     # Check for diminutive suffixes
#     features['DIMINUTIVE'] = 1 if re.search(r'ок$|ек$|ик$|очк$', word) else 0

#     # Check last 4 letters for reflexive verbs
#     features['REFLEXIVE'] = 1 if re.search(r'ся$|сь$', word) else 0
    
#     # Check for punctuation
#     features['has_punctuation'] = int(any(char in word for char in r'!?.,"-')) 
    
#     return features

## Use K nearest neighbours to tag

In [231]:
# from collections import defaultdict

# # Calculate euclidean distance between training feature vector and test feature vector
# def euclidean_distance(a, b):
#     return (sum((a[x1] - b[x2])**2 for x1, x2 in zip(a, b)))**0.5

# # Build training set
# def build_training_set(sentences, get_features_func):
#     X, y = [], []
#     for sentence in sentences:
#         for word, tag in sentence:
#             X.append(get_features_func(word))
#             y.append(tag)
#     return X, y
        
# # kNN model, default k = 5
# def predict(training_set, word, get_features_func, k=5):
#     input_features = get_features_func(word)
#     distances = defaultdict(float)
#     X, y = build_training_set(training_set, get_features_func)

#     # Compute distances between X_test and all training points
#     dist = [(euclidean_distance(input_features, x_train), y) 
#             for (x_train, y) in zip(X, y)]
    
#     # Sort distances and take top k
#     nearest = sorted(dist)[:k]

#     # Count votes for each class
#     votes = [y for _, y in nearest]
#     prediction = max(set(votes), key=votes.count)
    
#     return prediction

## CSV writer

In [232]:
# import csv
# def lists_to_csv(list_a, list_b, filename):
#     with open(filename, 'w', encoding='utf-8') as file:
#         for a, b in zip(list_a, list_b):
#             file.write(f'{a} {b}\n')

In [233]:
filtered_rows = ES_train[ES_train['word'] == 'iluminación']
filtered_rows

,word,label
349,iluminación,B-negative
2101,iluminación,B-neutral
2767,iluminación,O
18153,iluminación,B-positive
24403,iluminación,B-neutral


In [234]:
q4_emission_parameter

{('Estuvimos', 'O'): 1.0,
 ('hace', 'O'): 1.0,
 ('poco', 'O'): 1.0,
 ('mi', 'O'): 1.0,
 ('pareja', 'O'): 1.0,
 ('y', 'O'): 0.9932104752667313,
 ('yo', 'O'): 0.972972972972973,
 ('comiendo', 'O'): 1.0,
 ('resultó', 'O'): 1.0,
 ('todo', 'O'): 0.9913793103448276,
 ('muy', 'O'): 0.9949748743718593,
 ('bien', 'O'): 0.9939759036144579,
 (',', 'O'): 0.9970041941282205,
 ('tanto', 'O'): 1.0,
 ('la', 'O'): 0.9792477302204928,
 ('comida', 'B-positive'): 0.7444933920704846,
 ('el', 'O'): 0.9907407407407407,
 ('vino', 'B-positive'): 0.15384615384615385,
 ('trato', 'B-positive'): 0.7333333333333333,
 ('decoración', 'B-positive'): 0.7777777777777778,
 ('…', 'O'): 0.9782608695652174,
 ('nos', 'O'): 1.0,
 ('gustó', 'O'): 1.0,
 ('mucho', 'O'): 0.9814814814814815,
 ('.', 'O'): 0.999384236453202,
 ('Por', 'O'): 1.0,
 ('poner', 'O'): 1.0,
 ('algún', 'O'): 1.0,
 ('pero', 'O'): 0.9896373056994818,
 ('quizá', 'O'): 1.0,
 ('jamón', 'B-negative'): 0.16666666666666666,
 ('no', 'O'): 0.9972972972972973,
 ('era',

## ES Test (dev)

In [235]:
# # Spanish Test
# es_list = ES_dev_in['word'].to_list()
# es_prediction = []
# for word in es_list:
#     es_prediction.append(predict(es_sentences, word, get_features_es))
# lists_to_csv(es_list, es_prediction, "./ES/dev.p4.out")

## RU Test (dev)

In [236]:
# # Russian Test
# ru_list = RU_dev_in['word'].to_list()
# ru_prediction = []
# for word in ru_list:
#     ru_prediction.append(predict(ru_sentences, word, get_features_ru))
# lists_to_csv(ru_list, ru_prediction, "./RU/dev.p4.out")

In [237]:
ES_train

,word,label
0,Estuvimos,O
1,hace,O
2,poco,O
3,mi,O
4,pareja,O
...,...,...
33028,con,O
33029,la,O
33030,carta,O
33031,.,O


In [238]:
def train_estimate_emission_parameters_word(train):
    word_count = {}
    emission_count = {}
    label_count = {}

    for row in range(train.shape[0]):
        word = train.iloc[row, 0]
        label = train.iloc[row, 1]
        if (word not in word_count) and (word != "null"):
            word_count[word] = 1
        elif ( word != "null" ):
            word_count[word] += 1
        
        if (label not in label_count) and (label != "null"):
            label_count[label] = 1
        elif (label != "null"):
            label_count[label] += 1

        if ((word, label) not in emission_count) and (word != "null"):
            emission_count[(word, label)] = 1
        elif (word != "null"):
            emission_count[(word, label)] += 1

    emission_parameter = {}
    for word, label in emission_count:
        emission_parameter[(word, label)] = emission_count[(word, label)] / word_count[word]

    return emission_parameter, emission_count, label_count

In [239]:
q4_emission_parameter, q4_emission_count, q4_label_count = train_estimate_emission_parameters_word(ES_train)

In [240]:
sum(q4_label_count.values())

31176

In [241]:
q4_emission_parameter

{('Estuvimos', 'O'): 1.0,
 ('hace', 'O'): 1.0,
 ('poco', 'O'): 1.0,
 ('mi', 'O'): 1.0,
 ('pareja', 'O'): 1.0,
 ('y', 'O'): 0.9932104752667313,
 ('yo', 'O'): 0.972972972972973,
 ('comiendo', 'O'): 1.0,
 ('resultó', 'O'): 1.0,
 ('todo', 'O'): 0.9913793103448276,
 ('muy', 'O'): 0.9949748743718593,
 ('bien', 'O'): 0.9939759036144579,
 (',', 'O'): 0.9970041941282205,
 ('tanto', 'O'): 1.0,
 ('la', 'O'): 0.9792477302204928,
 ('comida', 'B-positive'): 0.7444933920704846,
 ('el', 'O'): 0.9907407407407407,
 ('vino', 'B-positive'): 0.15384615384615385,
 ('trato', 'B-positive'): 0.7333333333333333,
 ('decoración', 'B-positive'): 0.7777777777777778,
 ('…', 'O'): 0.9782608695652174,
 ('nos', 'O'): 1.0,
 ('gustó', 'O'): 1.0,
 ('mucho', 'O'): 0.9814814814814815,
 ('.', 'O'): 0.999384236453202,
 ('Por', 'O'): 1.0,
 ('poner', 'O'): 1.0,
 ('algún', 'O'): 1.0,
 ('pero', 'O'): 0.9896373056994818,
 ('quizá', 'O'): 1.0,
 ('jamón', 'B-negative'): 0.16666666666666666,
 ('no', 'O'): 0.9972972972972973,
 ('era',

In [242]:
q4_emission_count

{('Estuvimos', 'O'): 6,
 ('hace', 'O'): 26,
 ('poco', 'O'): 55,
 ('mi', 'O'): 72,
 ('pareja', 'O'): 13,
 ('y', 'O'): 1024,
 ('yo', 'O'): 36,
 ('comiendo', 'O'): 10,
 ('resultó', 'O'): 4,
 ('todo', 'O'): 115,
 ('muy', 'O'): 396,
 ('bien', 'O'): 165,
 (',', 'O'): 1664,
 ('tanto', 'O'): 39,
 ('la', 'O'): 755,
 ('comida', 'B-positive'): 169,
 ('el', 'O'): 642,
 ('vino', 'B-positive'): 6,
 ('trato', 'B-positive'): 44,
 ('decoración', 'B-positive'): 7,
 ('…', 'O'): 45,
 ('nos', 'O'): 146,
 ('gustó', 'O'): 11,
 ('mucho', 'O'): 53,
 ('.', 'O'): 1623,
 ('Por', 'O'): 30,
 ('poner', 'O'): 5,
 ('algún', 'O'): 6,
 ('pero', 'O'): 191,
 ('quizá', 'O'): 4,
 ('jamón', 'B-negative'): 1,
 ('no', 'O'): 369,
 ('era', 'O'): 52,
 ('lo', 'O'): 270,
 ('"', 'O'): 39,
 ('ibérico', 'O'): 2,
 ('que', 'O'): 845,
 ('cabía', 'O'): 1,
 ('esperar', 'O'): 5,
 ('Bien', 'O'): 6,
 ('sabe', 'O'): 5,
 ('autor', 'O'): 2,
 ('del', 'O'): 122,
 ('blog', 'O'): 2,
 (')', 'O'): 131,
 ('Comida', 'B-positive'): 13,
 ('exquisita', 'O'

In [243]:
for key, value in q4_emission_count.items():
    if key[0] == 'y':
        print(key, value)

('y', 'O') 1024
('y', 'I-negative') 2
('y', 'I-positive') 5


In [244]:
def upgraded_estimate_emission_parameters_word(test_set, emission_parameter, emission_counts, label_counts):
    # predict the label of the test set
    k = 0
    special_word = '#UNK#'
    test_copy = test_set.copy()
    #train word list is all the first argument in keys of emission counts
    train_word_list= [word for word, label in emission_counts.keys()]
    for i in range(len(test_copy["word"])):
        word = test_copy["word"][i]
        if word not in train_word_list:
            test_copy["word"][i] = special_word
            k+=1

    for word in test_copy["word"]:
        for label in label_counts:
            if (word == special_word) and ((word, label) not in emission_parameter):
                emission_parameter[(word, label)] = (k*(label_counts.get(label)/sum(label_counts.values())))/k
                # print((word, label), emission_parameter.get((word, label)))
                      
    return emission_parameter

In [245]:
q4_ES_emit, q4_ES_emitcount, q4_ES_label_count = train_estimate_emission_parameters_word(ES_train)
q4_ES_emit_fin = upgraded_estimate_emission_parameters_word(ES_dev_in, q4_ES_emit, q4_ES_emitcount, q4_ES_label_count)

q4_RU_emit, q4_RU_emitcount, q4_RU_label_count = train_estimate_emission_parameters_word(RU_train)
q4_RU_emit_fin = upgraded_estimate_emission_parameters_word(RU_dev_in, q4_RU_emit, q4_RU_emitcount, q4_RU_label_count)


In [246]:
ES_pred_q4 = predict(q4_ES_emit_fin, ES_transit, ES_seq_list)
RU_pred_q4 = predict(q4_RU_emit_fin, RU_transit, RU_seq_list)

C:\Users\limyo\AppData\Local\Temp\ipykernel_4404\4078816607.py:14: RuntimeWarning: divide by zero encountered in log
  a = np.log(transit[f"{v}|{u}"])


In [247]:
ES_pred_q4

[{(0, 'START'): 0,
  (1, 'O'): -3.5164221289006354,
  (1, 'B-negative'): -3.3778500176651645,
  (2, 'O'): -4.385025163275429,
  (2, 'I-positive'): -inf,
  (2, 'I-negative'): -6.551297667598213,
  (3, 'O'): -4.506413795820002,
  (4, 'O'): -4.627802428364575,
  (5, 'O'): -4.749191060909148,
  (6, 'O'): -4.87057969345372,
  (7, 'O'): -5.077247243700501,
  (7, 'I-neutral'): -inf,
  (7, 'I-positive'): -inf,
  (7, 'I-negative'): -inf,
  (8, 'O'): -5.198635876245074,
  (9, 'O'): -5.323024811070328,
  (9, 'I-positive'): -inf,
  (9, 'I-negative'): -inf,
  (10, 'O'): -5.454830204473157,
  (10, 'I-neutral'): -inf,
  (10, 'B-positive'): -10.67586609696951,
  (11, 'O'): -5.582261151473692,
  (11, 'B-positive'): -10.656969089823997,
  (12, 'O'): -5.703649784018265,
  (13, 'O'): -5.82565436976987,
  (13, 'I-positive'): -inf},
 {(0, 'START'): 0,
  (1, 'B-negative'): -4.8182116000553314,
  (1, 'B-positive'): -3.676703429954209,
  (1, 'O'): -3.36981865470876,
  (2, 'O'): -3.491207287253333,
  (3, 'O'): 

In [ ]:
ES_proc_q4 = processing_pred(ES_pred_q4)
RU_proc_q4 = processing_pred(RU_pred_q4)

## Evaluate

In [ ]:
import subprocess

command3 = ['python', r'.\ES\evalResult.py', r'.\ES\dev.out', r'./ES/dev.p4.out']
result3 = subprocess.run(command3, stdout=subprocess.PIPE)
print("ES result is:")
print(result3.stdout.decode())

command3 = ['python', r'.\RU\evalResult.py', r'.\RU\dev.out', r'./RU/dev.p4.out']
result3 = subprocess.run(command3, stdout=subprocess.PIPE)
print("RU result is:")
print(result3.stdout.decode())

ES result is:

#Entity in gold data: 229
#Entity in prediction: 4057

#Correct Entity : 0
Entity  precision: 0.0000
Entity  recall: 0.0000
Entity  F: 0.0000

#Correct Sentiment : 0
Sentiment  precision: 0.0000
Sentiment  recall: 0.0000
Sentiment  F: 0.0000

RU result is:



## Import test files

In [ ]:
with open('./ES/test.in', 'r', encoding = 'utf8') as f:
    r2 = [lines.rstrip("\n") for lines in f.readlines()]
ES_test_in_tup = [ lines if lines != "" else "null" for lines in r2]
ES_test_in = pd.DataFrame(data = ES_test_in_tup, columns = ["word"])
es_test_list = ES_test_in['word'].to_list()

with open('./RU/test.in', 'r', encoding = 'utf8') as f:
    r2 = [lines.rstrip("\n") for lines in f.readlines()]
RU_test_in_tup = [ lines if lines != "" else "null" for lines in r2]
RU_test_in = pd.DataFrame(data = RU_test_in_tup, columns = ["word"])
ru_test_list = RU_test_in['word'].to_list()

## ES Test (final)

In [ ]:
# Spanish Test
es_test_prediction = []
for word in es_test_list:
    es_test_prediction.append(predict(es_sentences, word, get_features_es))
lists_to_csv(es_test_list, es_test_prediction, "./ES/test.p4.out")

## RU Test (final)

In [ ]:
# Russian Test
ru_test_prediction = []
for word in ru_test_list:
    ru_test_prediction.append(predict(ru_sentences, word, get_features_ru))
lists_to_csv(ru_test_list, ru_test_prediction, "./RU/test.p4.out")